In [68]:
import pandas as pd

### 1. Clean Real Demand Data

##### September_2025_data

In [2]:
# Load the files
real = pd.read_csv("./RealDemand_Sept2025.csv", sep=';')
real.head()

,id,name,geoid,geoname,value,datetime
0,1293,Real demand,NaN,NaN,23820.000,2025-09-01T00:00:00+02:00
1,1293,Real demand,NaN,NaN,22533.583,2025-09-01T01:00:00+02:00
2,1293,Real demand,NaN,NaN,21515.000,2025-09-01T02:00:00+02:00
3,1293,Real demand,NaN,NaN,20827.833,2025-09-01T03:00:00+02:00
4,1293,Real demand,NaN,NaN,20696.667,2025-09-01T04:00:00+02:00


In [3]:
# Extract datetime and value
real["datetime"] = pd.to_datetime(real["datetime"])
real = real[["datetime", "value"]].rename(columns={"value": "NLD"})

# Format to match demand.csv
# First two header rows as in demand.csv
header = pd.DataFrame({"nodes": ["techs"], "NLD": ["demand_power"]})

# Prepare final formatted time series
ts = pd.DataFrame({
    "nodes": real["datetime"].dt.strftime("%Y-%m-%d %H:%M:%S"),
    "NLD": real["NLD"]
})

# Combine them together
real_demand = pd.concat([header, ts], ignore_index=True)

# Save to new CSV
real_demand.to_csv("./demand_sept2025.csv", index=False)

real_demand.head(10)


,nodes,NLD
0,techs,demand_power
1,2025-09-01 00:00:00,23820.0
2,2025-09-01 01:00:00,22533.583
3,2025-09-01 02:00:00,21515.0
4,2025-09-01 03:00:00,20827.833
5,2025-09-01 04:00:00,20696.667
6,2025-09-01 05:00:00,21107.833
7,2025-09-01 06:00:00,23696.917
8,2025-09-01 07:00:00,26555.25
9,2025-09-01 08:00:00,27857.25


##### 2024_year_data

In [103]:
# Load the files
real_year = pd.read_csv("./RealDemand_2024.csv", sep=';')
real_year.head()

,id,name,geoid,geoname,value,datetime
0,1293,Real demand,NaN,NaN,21863.416667,2024-01-01T00:00:00+01:00
1,1293,Real demand,NaN,NaN,21200.666667,2024-01-01T01:00:00+01:00
2,1293,Real demand,NaN,NaN,20033.083333,2024-01-01T02:00:00+01:00
3,1293,Real demand,NaN,NaN,18867.666667,2024-01-01T03:00:00+01:00
4,1293,Real demand,NaN,NaN,18137.166667,2024-01-01T04:00:00+01:00


In [104]:
# Parse datetimes; handle timezone-aware strings by parsing with utc=True
real_year["datetime"] = pd.to_datetime(real_year["datetime"], errors="coerce", utc=True)
real_year["datetime"] = real_year["datetime"].dt.tz_convert("Europe/Madrid").dt.tz_localize(None)
real_year.head()

,id,name,geoid,geoname,value,datetime
0,1293,Real demand,NaN,NaN,21863.416667,2024-01-01 00:00:00
1,1293,Real demand,NaN,NaN,21200.666667,2024-01-01 01:00:00
2,1293,Real demand,NaN,NaN,20033.083333,2024-01-01 02:00:00
3,1293,Real demand,NaN,NaN,18867.666667,2024-01-01 03:00:00
4,1293,Real demand,NaN,NaN,18137.166667,2024-01-01 04:00:00


In [105]:
real_year = real_year[["datetime", "value"]].rename(columns={"value": "NLD"})
real_year.head()

,datetime,NLD
0,2024-01-01 00:00:00,21863.416667
1,2024-01-01 01:00:00,21200.666667
2,2024-01-01 02:00:00,20033.083333
3,2024-01-01 03:00:00,18867.666667
4,2024-01-01 04:00:00,18137.166667


In [106]:
real_year.tail()

,datetime,NLD
8779,2024-12-31 19:00:00,31136.333333
8780,2024-12-31 20:00:00,30291.666667
8781,2024-12-31 21:00:00,28500.250000
8782,2024-12-31 22:00:00,25583.500000
8783,2024-12-31 23:00:00,24053.416667


In [107]:
# delete rows of last day (2024-12-31)
real_year = real_year[real_year["datetime"].dt.date != pd.to_datetime("2024-12-31").date()]
real_year.tail()

,datetime,NLD
8755,2024-12-30 19:00:00,32269.333333
8756,2024-12-30 20:00:00,32806.250000
8757,2024-12-30 21:00:00,32516.666667
8758,2024-12-30 22:00:00,30309.833333
8759,2024-12-30 23:00:00,27405.333333


In [108]:
# delete rows with same datetime, keep the first occurrence
real_year = real_year.drop_duplicates(subset=["datetime"], keep="first")

In [109]:
# Format to match demand.csv
# First two header rows as in demand.csv
header = pd.DataFrame({"nodes": ["techs"], "NLD": ["demand_power"]})

# Prepare final formatted time series
ts = pd.DataFrame({
    "nodes": real_year["datetime"].dt.strftime("%Y-%m-%d %H:%M:%S"),
    "NLD": real_year["NLD"]
})

# Combine them together
real_demand_year = pd.concat([header, ts], ignore_index=True)

# Save to new CSV
real_demand_year.to_csv("./demand_year.csv", index=False)

real_demand_year.head(10)

,nodes,NLD
0,techs,demand_power
1,2024-01-01 00:00:00,21863.416667
2,2024-01-01 01:00:00,21200.666667
3,2024-01-01 02:00:00,20033.083333
4,2024-01-01 03:00:00,18867.666667
5,2024-01-01 04:00:00,18137.166667
6,2024-01-01 05:00:00,17881.166667
7,2024-01-01 06:00:00,18059.166667
8,2024-01-01 07:00:00,18429.333333
9,2024-01-01 08:00:00,18439.333333


In [110]:
# find the mean of NLD
mean_demand = real_year["NLD"].mean()
mean_demand

26442.079289873273

In [125]:
demand_2030 = 35447
ratio = demand_2030 / mean_demand

# scale the NLD values
real_2030 = real_year.copy()
real_2030["NLD"] = real_2030["NLD"] * ratio

real_2030.head()

,datetime,NLD
0,2024-01-01 00:00:00,29309.061594
1,2024-01-01 01:00:00,28420.610312
2,2024-01-01 02:00:00,26855.403357
3,2024-01-01 03:00:00,25293.100932
4,2024-01-01 04:00:00,24313.827206


In [126]:
header = pd.DataFrame({"nodes": ["techs"], "NLD": ["demand_power"]})

# Prepare final formatted time series
ts = pd.DataFrame({
    "nodes": real_2030["datetime"].dt.strftime("%Y-%m-%d %H:%M:%S"),
    "NLD": real_2030["NLD"]
})

# Combine them together
real_demand_2030 = pd.concat([header, ts], ignore_index=True)

# Save to new CSV
real_demand_2030.to_csv("./demand_2030.csv", index=False)

real_demand_2030.head()

,nodes,NLD
0,techs,demand_power
1,2024-01-01 00:00:00,29309.061594
2,2024-01-01 01:00:00,28420.610312
3,2024-01-01 02:00:00,26855.403357
4,2024-01-01 03:00:00,25293.100932


### 2. Clean Renewables Data

##### September_2025_data

In [20]:
# load solar data
solar = pd.read_csv("./ForecastGenerationSolar_Sept2025.csv", sep=';')
solar.tail()

,id,name,geoid,geoname,value,datetime
714,10034,Forecast generation Solar,NaN,NaN,3686.00,2025-09-30T19:00:00+02:00
715,10034,Forecast generation Solar,NaN,NaN,357.00,2025-09-30T20:00:00+02:00
716,10034,Forecast generation Solar,NaN,NaN,238.25,2025-09-30T21:00:00+02:00
717,10034,Forecast generation Solar,NaN,NaN,315.75,2025-09-30T22:00:00+02:00
718,10034,Forecast generation Solar,NaN,NaN,291.75,2025-09-30T23:00:00+02:00


In [21]:
# load wind data
wind = pd.read_csv("./MeasuredOnshoreWindGeneration_Sept2025.csv", sep=';')
wind.tail()

,id,name,geoid,geoname,value,datetime
715,1159,Measured Onshore wind generation,NaN,NaN,2857.048,2025-09-30T19:00:00+02:00
716,1159,Measured Onshore wind generation,NaN,NaN,3235.416,2025-09-30T20:00:00+02:00
717,1159,Measured Onshore wind generation,NaN,NaN,4015.845,2025-09-30T21:00:00+02:00
718,1159,Measured Onshore wind generation,NaN,NaN,4610.186,2025-09-30T22:00:00+02:00
719,1159,Measured Onshore wind generation,NaN,NaN,4986.580,2025-09-30T23:00:00+02:00


In [22]:
# add one column 'installed_capacity' with value 38659.3
solar["installed_capacity"] = 38659.3

# add one column 'CF_solar' as capacity factor, calculated as value / installed_capacity
solar["CF_solar"] = solar["value"] / solar["installed_capacity"]
solar.head()

,id,name,geoid,geoname,value,datetime,installed_capacity,CF_solar
0,10034,Forecast generation Solar,NaN,NaN,393.75,2025-09-01T00:00:00+02:00,38659.3,0.010185
1,10034,Forecast generation Solar,NaN,NaN,155.50,2025-09-01T01:00:00+02:00,38659.3,0.004022
2,10034,Forecast generation Solar,NaN,NaN,166.25,2025-09-01T02:00:00+02:00,38659.3,0.004300
3,10034,Forecast generation Solar,NaN,NaN,157.00,2025-09-01T03:00:00+02:00,38659.3,0.004061
4,10034,Forecast generation Solar,NaN,NaN,152.25,2025-09-01T04:00:00+02:00,38659.3,0.003938


In [23]:
# add one column 'installed_capacity' with value 33010.6
wind["installed_capacity"] = 33010.6

# add one column 'CF_wind' as capacity factor, calculated as value / installed_capacity
wind["CF_wind"] = wind["value"] / wind["installed_capacity"]
wind.head()

,id,name,geoid,geoname,value,datetime,installed_capacity,CF_wind
0,1159,Measured Onshore wind generation,NaN,NaN,11362.571,2025-09-01T00:00:00+02:00,33010.6,0.344210
1,1159,Measured Onshore wind generation,NaN,NaN,10791.856,2025-09-01T01:00:00+02:00,33010.6,0.326921
2,1159,Measured Onshore wind generation,NaN,NaN,10217.945,2025-09-01T02:00:00+02:00,33010.6,0.309535
3,1159,Measured Onshore wind generation,NaN,NaN,9947.512,2025-09-01T03:00:00+02:00,33010.6,0.301343
4,1159,Measured Onshore wind generation,NaN,NaN,9868.246,2025-09-01T04:00:00+02:00,33010.6,0.298942


In [49]:
# create final dataframe with datetime, CF_solar, CF_wind
renewables_df = pd.DataFrame({
    "datetime": solar["datetime"],
    "CF_solar": solar["CF_solar"],
    "CF_wind": wind["CF_wind"]
})

renewables_df.head()

,datetime,CF_solar,CF_wind
0,2025-09-01T00:00:00+02:00,0.010185,0.344210
1,2025-09-01T01:00:00+02:00,0.004022,0.326921
2,2025-09-01T02:00:00+02:00,0.004300,0.309535
3,2025-09-01T03:00:00+02:00,0.004061,0.301343
4,2025-09-01T04:00:00+02:00,0.003938,0.298942


In [50]:
# Extract datetime and value
renewables_df["datetime"] = pd.to_datetime(renewables_df["datetime"])
renewables_df = renewables_df[["datetime", "CF_solar", "CF_wind"]].rename(columns={"CF_solar": "NLD", "CF_wind": "NLD1"})

# Format to match demand.csv
# First header rows as in demand.csv
header = pd.DataFrame({"nodes": ["techs"], "NLD": ["solar_pv"], "NLD1": ["wind_onshore"]})

# Prepare final formatted time series
ts = pd.DataFrame({
    "nodes": renewables_df["datetime"].dt.strftime("%Y-%m-%d %H:%M:%S"),
    "NLD": renewables_df["NLD"],
    "NLD1": renewables_df["NLD1"]
})

# Combine them together
renewables_final = pd.concat([header, ts], ignore_index=True)

# change the name of NLD1 to NLD
renewables_final = renewables_final.rename(columns={"NLD1": "NLD"})

# delete last row of renewables_final
renewables_final = renewables_final[:-1]

# Save to new CSV
renewables_final.to_csv("./renewables_sept2025.csv", index=False)

renewables_final.tail()

,nodes,NLD,NLD
715,2025-09-30 19:00:00,0.095346,0.079022
716,2025-09-30 20:00:00,0.009235,0.086549
717,2025-09-30 21:00:00,0.006163,0.098011
718,2025-09-30 22:00:00,0.008168,0.121653
719,2025-09-30 23:00:00,0.007547,0.139658


##### October2024_September2025_year_data

In [35]:
# load solar data
solar_year = pd.read_csv("./ForecastGenerationSolar_2425.csv", sep=';')
solar_year.head()

,id,name,geoid,geoname,value,datetime
0,10034,Forecast generation Solar,NaN,NaN,519.250,2024-10-01T00:00:00+02:00
1,10034,Forecast generation Solar,NaN,NaN,500.700,2024-10-01T01:00:00+02:00
2,10034,Forecast generation Solar,NaN,NaN,465.825,2024-10-01T02:00:00+02:00
3,10034,Forecast generation Solar,NaN,NaN,434.225,2024-10-01T03:00:00+02:00
4,10034,Forecast generation Solar,NaN,NaN,313.325,2024-10-01T04:00:00+02:00


In [36]:
# load wind data
wind_year = pd.read_csv("./MeasuredOnshoreWindGeneration_2425.csv", sep=';')
wind_year.head()

,id,name,geoid,geoname,value,datetime
0,1159,Measured Onshore wind generation,NaN,NaN,4481.385,2024-10-01T00:00:00+02:00
1,1159,Measured Onshore wind generation,NaN,NaN,4823.612,2024-10-01T01:00:00+02:00
2,1159,Measured Onshore wind generation,NaN,NaN,5145.524,2024-10-01T02:00:00+02:00
3,1159,Measured Onshore wind generation,NaN,NaN,5508.119,2024-10-01T03:00:00+02:00
4,1159,Measured Onshore wind generation,NaN,NaN,5927.815,2024-10-01T04:00:00+02:00


In [37]:
# add one column 'installed_capacity' with value 38659.3
solar_year["installed_capacity"] = 38659.3

# add one column 'CF_solar' as capacity factor, calculated as value / installed_capacity
solar_year["CF_solar"] = solar_year["value"] / solar_year["installed_capacity"]
solar_year.head()

,id,name,geoid,geoname,value,datetime,installed_capacity,CF_solar
0,10034,Forecast generation Solar,NaN,NaN,519.250,2024-10-01T00:00:00+02:00,38659.3,0.013431
1,10034,Forecast generation Solar,NaN,NaN,500.700,2024-10-01T01:00:00+02:00,38659.3,0.012952
2,10034,Forecast generation Solar,NaN,NaN,465.825,2024-10-01T02:00:00+02:00,38659.3,0.012049
3,10034,Forecast generation Solar,NaN,NaN,434.225,2024-10-01T03:00:00+02:00,38659.3,0.011232
4,10034,Forecast generation Solar,NaN,NaN,313.325,2024-10-01T04:00:00+02:00,38659.3,0.008105


In [38]:
# add one column 'installed_capacity' with value 33010.6
wind_year["installed_capacity"] = 33010.6

# add one column 'CF_wind' as capacity factor, calculated as value / installed_capacity
wind_year["CF_wind"] = wind_year["value"] / wind_year["installed_capacity"]
wind_year.head()

,id,name,geoid,geoname,value,datetime,installed_capacity,CF_wind
0,1159,Measured Onshore wind generation,NaN,NaN,4481.385,2024-10-01T00:00:00+02:00,33010.6,0.135756
1,1159,Measured Onshore wind generation,NaN,NaN,4823.612,2024-10-01T01:00:00+02:00,33010.6,0.146123
2,1159,Measured Onshore wind generation,NaN,NaN,5145.524,2024-10-01T02:00:00+02:00,33010.6,0.155875
3,1159,Measured Onshore wind generation,NaN,NaN,5508.119,2024-10-01T03:00:00+02:00,33010.6,0.166859
4,1159,Measured Onshore wind generation,NaN,NaN,5927.815,2024-10-01T04:00:00+02:00,33010.6,0.179573


In [39]:
# create final dataframe with datetime, CF_solar, CF_wind
renewables_df_year = pd.DataFrame({
    "datetime": solar_year["datetime"],
    "CF_solar": solar_year["CF_solar"],
    "CF_wind": wind_year["CF_wind"]
})

renewables_df_year.head()

,datetime,CF_solar,CF_wind
0,2024-10-01T00:00:00+02:00,0.013431,0.135756
1,2024-10-01T01:00:00+02:00,0.012952,0.146123
2,2024-10-01T02:00:00+02:00,0.012049,0.155875
3,2024-10-01T03:00:00+02:00,0.011232,0.166859
4,2024-10-01T04:00:00+02:00,0.008105,0.179573


In [40]:
# Parse datetimes; handle timezone-aware strings by parsing with utc=True
renewables_df_year["datetime"] = pd.to_datetime(renewables_df_year["datetime"], errors="coerce", utc=True)
renewables_df_year["datetime"] = renewables_df_year["datetime"].dt.tz_convert("Europe/Madrid").dt.tz_localize(None)
renewables_df_year.head()

,datetime,CF_solar,CF_wind
0,2024-10-01 00:00:00,0.013431,0.135756
1,2024-10-01 01:00:00,0.012952,0.146123
2,2024-10-01 02:00:00,0.012049,0.155875
3,2024-10-01 03:00:00,0.011232,0.166859
4,2024-10-01 04:00:00,0.008105,0.179573


### New data clean

### Solar capacity factor data

In [77]:
# load PV capacity factor data from CSV
solar_cf = pd.read_csv("./PV_CF_2024.csv")
solar_cf.head()

C:\Users\yaowa\AppData\Local\Temp\ipykernel_11704\2381211353.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  solar_cf = pd.read_csv("./PV_CF_2024.csv")


,"# Renewables.ninja pv (country zones, hourly, 1980 - 2024) - ES - ninja-pv-countryzones-ES-hourly-merra2.csv.gz - Version: 1.4 - License: https://creativecommons.org/licenses/by-nc/4.0/ - Reference: https://doi.org/10.1016/j.energy.2016.08.060",Unnamed: 1
0,"# Units: time in UTC, other columns are bias-c...",NaN
1,"# {'units': {'time': 'UTC', 'NATIONAL': 'bias-...",NaN
2,time,NATIONAL
3,1980-01-01 00:00:00+00:00,0.000000
4,1980-01-01 01:00:00+00:00,0.000000


In [78]:
# drop first three rows
solar_cf = solar_cf.iloc[3:].reset_index(drop=True)

# change column names to datetime and CF_solar
solar_cf.columns = ["datetime", "CF_solar"]

# filter datatime between 2024-01-01 and 2024-12-31
solar_cf = solar_cf[(solar_cf["datetime"] >= "2024-01-01") & (solar_cf["datetime"] <= "2024-12-31")].reset_index(drop=True)
solar_cf.head()

,datetime,CF_solar
0,2024-01-01 00:00:00+00:00,0.0
1,2024-01-01 01:00:00+00:00,0.0
2,2024-01-01 02:00:00+00:00,0.0
3,2024-01-01 03:00:00+00:00,0.0
4,2024-01-01 04:00:00+00:00,0.0


### Wind capacity factor data

In [79]:
# load wind onshore capacity factor data from CSV
windonshore_cf = pd.read_csv("./WindOnshore_CF_2024.csv")
windonshore_cf.head()

C:\Users\yaowa\AppData\Local\Temp\ipykernel_11704\4013043967.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  windonshore_cf = pd.read_csv("./WindOnshore_CF_2024.csv")


,"# Renewables.ninja Wind (current onshore fleet, country zones, hourly, 1980 - 2024) - Spain - ninja_wind_countryzones_hourly_ES_current-onshore_merra2.csv - Version 1.4 - License: https://creativecommons.org/licenses/by-nc/4.0/ - Reference: https://doi.org/10.1016/j.energy.2016.08.068",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,# Units: time in UTC; other columns are capaci...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"# {'units': {'time': 'UTC', 'NATIONAL': 'bias-...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,time,NATIONAL,ES11,ES12,ES13,ES21,ES22,ES23,ES24,ES30,...,ES42,ES43,ES51,ES52,ES53,ES61,ES62,ES63,ES64,ES70
3,1980-01-01 00:00:00+00:00,0.460166,0.409802,0.490627,0.685976,0.453296,0.279961,0.476468,0.485619,0.29944,...,0.48794,0.211598,0.124726,0.563356,0.10355,0.270761,0.48617,0.205465,0.6481,0.390197
4,1980-01-01 01:00:00+00:00,0.483027,0.372898,0.448044,0.713033,0.520734,0.408829,0.592286,0.583023,0.286953,...,0.459711,0.216962,0.162608,0.597692,0.094159,0.267149,0.504723,0.21085,0.611547,0.377099


In [80]:
# drop first three rows, and keep only first two columns
windonshore_cf = windonshore_cf.iloc[3:, :2].reset_index(drop=True)

# change column names to datetime and CF_solar
windonshore_cf.columns = ["datetime", "CF_windonshore"]

# filter datatime between 2024-01-01 and 2024-12-31
windonshore_cf = windonshore_cf[(windonshore_cf["datetime"] >= "2024-01-01") & (windonshore_cf["datetime"] <= "2024-12-31")].reset_index(drop=True)
windonshore_cf.head()

,datetime,CF_windonshore
0,2024-01-01 00:00:00+00:00,0.231344
1,2024-01-01 01:00:00+00:00,0.222525
2,2024-01-01 02:00:00+00:00,0.214007
3,2024-01-01 03:00:00+00:00,0.204727
4,2024-01-01 04:00:00+00:00,0.196597


In [81]:
# load wind offshore capacity factor data from CSV
windoffshore_cf = pd.read_csv("./WindOffshore_CF_2024.csv")
windoffshore_cf.head()

C:\Users\yaowa\AppData\Local\Temp\ipykernel_11704\2996865825.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  windoffshore_cf = pd.read_csv("./WindOffshore_CF_2024.csv")


,"# Renewables.ninja Wind (current offshore fleet, country zones, hourly, 1980 - 2024) - Spain - ninja_wind_countryzones_hourly_ES_current-offshore_merra2.csv - Version 1.4 - License: https://creativecommons.org/licenses/by-nc/4.0/ - Reference: https://doi.org/10.1016/j.energy.2016.08.068",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,# Units: time in UTC; other columns are capaci...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"# {'units': {'time': 'UTC', 'NATIONAL': 'bias-...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,time,NATIONAL,ES11,ES12,ES13,ES21,ES22,ES23,ES24,ES30,...,ES42,ES43,ES51,ES52,ES53,ES61,ES62,ES63,ES64,ES70
3,1980-01-01 00:00:00+00:00,0.917527,NaN,NaN,NaN,0.917528,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1980-01-01 01:00:00+00:00,0.970495,NaN,NaN,NaN,0.970497,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
# drop first three rows, and keep only first two columns
windoffshore_cf = windoffshore_cf.iloc[3:, :2].reset_index(drop=True)

# change column names to datetime and CF_solar
windoffshore_cf.columns = ["datetime", "CF_windoffshore"]

# filter datatime between 2024-01-01 and 2024-12-31
windoffshore_cf = windoffshore_cf[(windoffshore_cf["datetime"] >= "2024-01-01") & (windoffshore_cf["datetime"] <= "2024-12-31")].reset_index(drop=True)
windoffshore_cf.head()

,datetime,CF_windoffshore
0,2024-01-01 00:00:00+00:00,0.7632
1,2024-01-01 01:00:00+00:00,0.776559
2,2024-01-01 02:00:00+00:00,0.742909
3,2024-01-01 03:00:00+00:00,0.673741
4,2024-01-01 04:00:00+00:00,0.6226


In [83]:
# merge solar_cf and windonshore_cf on datetime
solar_wind_2024 = pd.merge(solar_cf, windonshore_cf, on="datetime")
solar_wind_2024 = pd.merge(solar_wind_2024, windoffshore_cf, on="datetime")
solar_wind_2024.head()

,datetime,CF_solar,CF_windonshore,CF_windoffshore
0,2024-01-01 00:00:00+00:00,0.0,0.231344,0.7632
1,2024-01-01 01:00:00+00:00,0.0,0.222525,0.776559
2,2024-01-01 02:00:00+00:00,0.0,0.214007,0.742909
3,2024-01-01 03:00:00+00:00,0.0,0.204727,0.673741
4,2024-01-01 04:00:00+00:00,0.0,0.196597,0.6226


### Hydro power 

In [84]:
# load hydro data
hydro = pd.read_csv("./HydroMonthly_2024.csv", sep=';')
hydro.head()

,id,name,geoid,geoname,value,datetime
0,472,Hydro UGH Available power generation capacity ...,NaN,NaN,13615.818414,2024-01-01T00:00:00+01:00
1,472,Hydro UGH Available power generation capacity ...,NaN,NaN,13532.739368,2024-02-01T00:00:00+01:00
2,472,Hydro UGH Available power generation capacity ...,NaN,NaN,13220.961373,2024-03-01T00:00:00+01:00
3,472,Hydro UGH Available power generation capacity ...,NaN,NaN,13257.650278,2024-04-01T00:00:00+02:00
4,472,Hydro UGH Available power generation capacity ...,NaN,NaN,12449.654704,2024-05-01T00:00:00+02:00


In [85]:
# add one column 'installed_capacity', 'capacity factor' calculated as value / installed_capacity
hydro["installed_capacity"] = 17077
hydro["CF_hydro"] = hydro["value"] / hydro["installed_capacity"]
hydro.head()


,id,name,geoid,geoname,value,datetime,installed_capacity,CF_hydro
0,472,Hydro UGH Available power generation capacity ...,NaN,NaN,13615.818414,2024-01-01T00:00:00+01:00,17077,0.797319
1,472,Hydro UGH Available power generation capacity ...,NaN,NaN,13532.739368,2024-02-01T00:00:00+01:00,17077,0.792454
2,472,Hydro UGH Available power generation capacity ...,NaN,NaN,13220.961373,2024-03-01T00:00:00+01:00,17077,0.774197
3,472,Hydro UGH Available power generation capacity ...,NaN,NaN,13257.650278,2024-04-01T00:00:00+02:00,17077,0.776345
4,472,Hydro UGH Available power generation capacity ...,NaN,NaN,12449.654704,2024-05-01T00:00:00+02:00,17077,0.729031


In [86]:
hydro_cf = hydro[["datetime", "CF_hydro"]]

# Parse datetimes; handle timezone-aware strings by parsing with utc=True
hydro_cf["datetime"] = pd.to_datetime(hydro_cf["datetime"], errors="coerce", utc=True)
hydro_cf["datetime"] = hydro_cf["datetime"].dt.tz_convert("Europe/Madrid").dt.tz_localize(None)
hydro_cf.head()

C:\Users\yaowa\AppData\Local\Temp\ipykernel_11704\1894073553.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hydro_cf["datetime"] = pd.to_datetime(hydro_cf["datetime"], errors="coerce", utc=True)
C:\Users\yaowa\AppData\Local\Temp\ipykernel_11704\1894073553.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hydro_cf["datetime"] = hydro_cf["datetime"].dt.tz_convert("Europe/Madrid").dt.tz_localize(None)


,datetime,CF_hydro
0,2024-01-01,0.797319
1,2024-02-01,0.792454
2,2024-03-01,0.774197
3,2024-04-01,0.776345
4,2024-05-01,0.729031


In [92]:
solar_wind_2024['year_month'] = pd.to_datetime(solar_wind_2024['datetime']).dt.to_period('M')
hydro_cf['year_month'] = pd.to_datetime(hydro_cf['datetime']).dt.to_period('M')

# Merge on year_month
renewables_2024 = pd.merge(
    solar_wind_2024, 
    hydro_cf[['year_month', 'CF_hydro']], 
    on='year_month', 
    how='left'
)

# Drop the temporary year_month column
renewables_2024 = renewables_2024.drop(columns=['year_month'])

renewables_2024.head()

C:\Users\yaowa\AppData\Local\Temp\ipykernel_11704\3538911047.py:1: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  solar_wind_2024['year_month'] = pd.to_datetime(solar_wind_2024['datetime']).dt.to_period('M')
C:\Users\yaowa\AppData\Local\Temp\ipykernel_11704\3538911047.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hydro_cf['year_month'] = pd.to_datetime(hydro_cf['datetime']).dt.to_period('M')


,datetime,CF_solar,CF_windonshore,CF_windoffshore,CF_hydro
0,2024-01-01 00:00:00+00:00,0.0,0.231344,0.7632,0.797319
1,2024-01-01 01:00:00+00:00,0.0,0.222525,0.776559,0.797319
2,2024-01-01 02:00:00+00:00,0.0,0.214007,0.742909,0.797319
3,2024-01-01 03:00:00+00:00,0.0,0.204727,0.673741,0.797319
4,2024-01-01 04:00:00+00:00,0.0,0.196597,0.6226,0.797319


In [93]:
renewables_2024.shape

(8760, 5)

In [94]:
# delete rows with same datetime, keep the first occurrence
renewables_df_year = renewables_2024.drop_duplicates(subset=["datetime"], keep="first")
renewables_df_year.shape

(8760, 5)

In [95]:
renewables_df_year = renewables_df_year[["datetime", "CF_solar", "CF_windonshore", "CF_windoffshore", "CF_hydro"]].rename(columns={"CF_solar": "NLD", "CF_windonshore": "NLD1", "CF_windoffshore": "NLD2", "CF_hydro": "NLD3"})
renewables_df_year.head()

,datetime,NLD,NLD1,NLD2,NLD3
0,2024-01-01 00:00:00+00:00,0.0,0.231344,0.7632,0.797319
1,2024-01-01 01:00:00+00:00,0.0,0.222525,0.776559,0.797319
2,2024-01-01 02:00:00+00:00,0.0,0.214007,0.742909,0.797319
3,2024-01-01 03:00:00+00:00,0.0,0.204727,0.673741,0.797319
4,2024-01-01 04:00:00+00:00,0.0,0.196597,0.6226,0.797319


In [96]:
renewables_df_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   datetime  8760 non-null   object 
 1   NLD       8760 non-null   object 
 2   NLD1      8760 non-null   object 
 3   NLD2      8760 non-null   object 
 4   NLD3      8760 non-null   float64
dtypes: float64(1), object(4)
memory usage: 342.3+ KB


In [97]:
renewables_df_year["datetime"] = pd.to_datetime(renewables_df_year["datetime"], errors="coerce")

In [98]:
renewables_df_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   datetime  8760 non-null   datetime64[ns, UTC]
 1   NLD       8760 non-null   object             
 2   NLD1      8760 non-null   object             
 3   NLD2      8760 non-null   object             
 4   NLD3      8760 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(1), object(3)
memory usage: 342.3+ KB


In [99]:
# Format to match demand.csv
# First header rows as in demand.csv
header = pd.DataFrame({"nodes": ["techs"], "NLD": ["solar_pv"], "NLD1": ["wind_onshore"], "NLD2": ["wind_offshore"], "NLD3": ["hydro"]})

# Prepare final formatted time series
ts = pd.DataFrame({
    "nodes": renewables_df_year["datetime"].dt.strftime("%Y-%m-%d %H:%M:%S"),
    "NLD": renewables_df_year["NLD"],
    "NLD1": renewables_df_year["NLD1"],
    "NLD2": renewables_df_year["NLD2"],
    "NLD3": renewables_df_year["NLD3"]
})

# Combine them together
renewables_final_year = pd.concat([header, ts], ignore_index=True)


renewables_final_year.head(10)

,nodes,NLD,NLD1,NLD2,NLD3
0,techs,solar_pv,wind_onshore,wind_offshore,hydro
1,2024-01-01 00:00:00,0.0,0.231344,0.7632,0.797319
2,2024-01-01 01:00:00,0.0,0.222525,0.776559,0.797319
3,2024-01-01 02:00:00,0.0,0.214007,0.742909,0.797319
4,2024-01-01 03:00:00,0.0,0.204727,0.673741,0.797319
5,2024-01-01 04:00:00,0.0,0.196597,0.6226,0.797319
6,2024-01-01 05:00:00,0.0,0.192137,0.630086,0.797319
7,2024-01-01 06:00:00,0.0,0.194536,0.645159,0.797319
8,2024-01-01 07:00:00,0.0,0.204508,0.63155,0.797319
9,2024-01-01 08:00:00,0.048,0.198421,0.622468,0.797319


In [100]:
# change the name of NLD1 to NLD
renewables_final_year = renewables_final_year.rename(columns={"NLD1": "NLD", "NLD2": "NLD", "NLD3": "NLD"})
renewables_final_year.head()

,nodes,NLD,NLD,NLD,NLD
0,techs,solar_pv,wind_onshore,wind_offshore,hydro
1,2024-01-01 00:00:00,0.0,0.231344,0.7632,0.797319
2,2024-01-01 01:00:00,0.0,0.222525,0.776559,0.797319
3,2024-01-01 02:00:00,0.0,0.214007,0.742909,0.797319
4,2024-01-01 03:00:00,0.0,0.204727,0.673741,0.797319


In [101]:
renewables_final_year.tail()

,nodes,NLD,NLD,NLD,NLD
8756,2024-12-30 19:00:00,0.0,0.09093,0.060125,0.792087
8757,2024-12-30 20:00:00,0.0,0.108659,0.104132,0.792087
8758,2024-12-30 21:00:00,0.0,0.121571,0.18936,0.792087
8759,2024-12-30 22:00:00,0.0,0.12688,0.301705,0.792087
8760,2024-12-30 23:00:00,0.0,0.131307,0.398101,0.792087


In [102]:
# Save to new CSV
renewables_final_year.to_csv("./renewables_year.csv", index=False)